In [ ]:
#!pip install wordcloud
#!pip install langdetect
#!pip install googletrans
#!pip install textblob
#!pip install spacy
#!python -m spacy download en
#!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_lg
#!pip install langid
#!pip install google_trans_new
#!pip uninstall googletrans

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS 
from langdetect import detect
from itertools import cycle
#import googletrans
#from googletrans import Translator
from google_trans_new import google_translator 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
from sklearn import preprocessing
#from textblob import TextBlob
#from textblob.translate import NotTranslated
import random
import operator
import math
import tqdm
import time
import spacy
import json
import langid 

### This section below contains  Useful Functions 
- As we find new functions, we will create them here.

In [ ]:
language = ["es", "de", "fr", "ar", "te", "hi", "ja", "fa", "sq", "bg", "nl", "gu", "ig", "kk", "mt", "ps"]

def data_augmentation_lang_translation(message, language, aug_range=1):
    augmented_messages = []
    if hasattr(message, "decode"):
        message = message.decode("utf-8")

        
    def request(lang_tgt, text):
        t = google_translator(timeout=15)
        translate_text = t.translate(text.strip(), lang_tgt)
        return translate_text


    for j in range(0,aug_range) :
            if __name__ == "__main__" :
              pool = ThreadPool(20) # Threads
              time1 = time.time()
              try:
                  lang_tgt = sr.choice(language)
                  func = partial(request, lang_tgt)
                  results = pool.map(func, message)
              except Exception as e:
                  raise e
              pool.close()
              pool.join()

              time2 = time.time()
              print("Translating %s Descriptions to %s, a total of %s s"%("placeholder",lang_tgt,time2 - time1))
                
              pool = ThreadPool(20) # Threads
              time1 = time.time()
              try:
                  lang_tgt = 'en'
                  func = partial(request, lang_tgt)
                  results = pool.map(func, message)
              except Exception as e:
                  raise e
              pool.close()
              pool.join()

              time2 = time.time()
              #print("Translating %s Descriptions, a total of %s s"%("placeholder",lang_tgt,time2 - time1))

              augmented_messages.append(str(results))

    return augmented_messages


#pool = ThreadPool(20)
#def f(a, b, c):
#    print("{} {} {}".format(a, b, c))

#def main():
#    iterable = [1, 2, 3, 4, 5]
#    a = "hi"
#    b = "there"
#    func = partial(f, a, b)
#    pool.map(func, iterable)
#    pool.close()
#    pool.join()

#if __name__ == "__main__":
#    main()

In [ ]:

## Tokenizing sentence into token for finding synonym.
def make_tokenizer(texts):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(texts)
    return t


## Function to find synonym of words 
spacy.prefer_gpu()
#nlp = spacy.load('en', parser=False)
#nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_md')
def check_lemma(t,w) :
    r = [d for d in t if (nlp(d.text)[0].lemma_ != nlp(w.text)[0].lemma_)]
    return r

def get_word_synonym(word):
  filtered_words = [w for w in word.vocab if (not w.lower_ in stop) and w.is_lower == word.is_lower and w.prob >= -20] ## (not w.lower_ in stop) and
  similarity = sorted(filtered_words, key=lambda w: word.similarity(w), reverse=True)
  filtered_similarity = check_lemma(similarity[:30], word)
  return filtered_similarity[:3]

## Function for augmenting data by replacing words with synonym using spaCy
## This might not be best best method compared to data augmentation using language translation


sr_spacy = random.SystemRandom()
split_pattern = re.compile(r'\s+')
def data_augmentation_spaCy(message, aug_range=1) :
    augmented_messages = []
    for j in range(0,aug_range) :
        new_message = ""
        for i in filter(None, split_pattern.split(message)) :
            new_message = new_message + " " + sr_spacy.choice(filtered_synonym.get(i,[i]))
        augmented_messages.append(new_message)
    return augmented_messages


def loadEmbeddingMatrix(typeToLoad, vocab_dict):
    import gensim.models.keyedvectors as word2vec
    import gc

    # load different embedding file from Kaggle depending on which embedding
    # matrix we are going to experiment with
    if (typeToLoad == "gloveTwitter50d"):
        EMBEDDING_FILE = 'embeddings\glove-twitter-27b-50d/glove.twitter.27B.50d.txt'
        embed_size = 50
    elif (typeToLoad == "word2vec"):
        word2vecDict = word2vec.KeyedVectors.load_word2vec_format( "embeddings\GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin", binary=True)
        embed_size = 300
    elif (typeToLoad == "fasttext"):
        EMBEDDING_FILE = 'embeddings\\fasttext/wiki.simple.vec'
        embed_size = 300
    elif (typeToLoad == "glove840B300D"):
        EMBEDDING_FILE = 'embeddings\glove.840B.300d/glove.840B.300d.txt'
        embed_size = 300
    elif (typeToLoad == "glove6B300D"):
        EMBEDDING_FILE = 'embeddings\glove.6B\glove.6B.300d.txt'
        embed_size = 300
    elif (typeToLoad == "paragram"):
        EMBEDDING_FILE = 'embeddings\paragram_300_sl999\paragram_300_sl999.txt'
        embed_size = 300
    elif (typeToLoad == "wikiNews"):
        EMBEDDING_FILE = "embeddings\wiki-news-300d-1M\wiki-news-300d-1M.vec"
        embed_size = 300

    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')

    if (typeToLoad in ["gloveTwitter50d", "fasttext"]):
        embeddings_index = dict()
        # Transfer the embedding weights into a dictionary by iterating through every line of the file.
        f = open(EMBEDDING_FILE)
        for line in f:
            # split up line into an indexed array
            values = line.rstrip().rsplit(' ')  # line.split()
            # first index is word
            word = values[0]
            # store the rest of the values in the array as a new array
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs  # 50 dimensions
        f.close()
    elif (typeToLoad in ["glove840B300D", "paragram", "glove6B300D"]):
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding='latin'))
    elif (typeToLoad in ["wikiNews"]):
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o) > 100)
    else:
        embeddings_index = dict()
        for word in word2vecDict.wv.vocab:
            embeddings_index[word] = word2vecDict.word_vec(word)
    print('Loaded %s word vectors.' % len(embeddings_index))

    gc.collect()
    # We get the mean and standard deviation of the embedding weights so that we could maintain the
    # same statistics for the rest of our own random generated weights.
    all_embs = np.stack(list(embeddings_index.values()))
    emb_mean, emb_std = all_embs.mean(), all_embs.std()

    nb_words = len(vocab_dict)
    # We are going to set the embedding size to the pretrained dimension as we are replicating it.
    # the size will be Number of Words in Vocab X Embedding Size
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    gc.collect()

    # With the newly created embedding matrix, we'll fill it up with the words that we have in both
    # our own dictionary and loaded pretrained embedding.
    embeddedCount = 0
    for word, i in vocab_dict.items():
        #i -= 1
        # then we see if this word is in glove's dictionary, if yes, get the corresponding weights
        embedding_vector = embeddings_index.get(word)
        # and store inside the embedding matrix that we will train later on.
        if embedding_vector is not None:
            try :
                embedding_matrix[i] = embedding_vector
                embeddedCount += 1
            except IndexError:
                pass
    print('total embedded:', embeddedCount, 'common words')

    del embeddings_index
    gc.collect()

    # finally, return the embedding matrix
    return embedding_matrix

##### We manually create the csv file from the excell and use pandas to read the csv.
- For some reason when we use the read_excel function, the number of NaN increase to 8 

In [ ]:
mydata = pd.read_csv('datasets/input_data_after_preprocessing.csv')
#mydata = pd.read_excel("datasets/input_data.xlsx")

In [ ]:
mydata.head()

### Data Augmentation using spaCy

In [ ]:
# load nltk's English stopwords as variable called 'stop' and don't find synonym of those words.
stop = nltk.corpus.stopwords.words('english')

In [ ]:
mydata.info()

In [ ]:
mydata['Combined Description Cleaned'] = mydata['Combined Description Cleaned'].astype(str)

tokenizer = make_tokenizer(mydata['Combined Description Cleaned'])    

#X = tokenizer.texts_to_sequences(mydata['Combined Description Cleaned'])

#from keras.preprocessing.sequence import pad_sequences
#X = pad_sequences(X, 70)

In [ ]:
mydata['Combined Description Cleaned']

In [ ]:
#X

In [ ]:
## Dictionary of word index
index_word = {}
for word in tokenizer.word_index.keys():
    index_word[tokenizer.word_index[word]] = word

In [ ]:
index_word

In [ ]:
## word list
words = [value for key, value in index_word.items()]

In [ ]:
words

In [ ]:
## Synonym dictionary
synonym_dict = {}

for word in words:
    #if (not check_oos(word)) :
        synonym_dict.update({word : tuple([w.lower_ for w in get_word_synonym(nlp.vocab[word])])})
        print(word, " : ", [w.lower_ for w in get_word_synonym(nlp.vocab[word])])

In [ ]:
synonym_dict

In [ ]:
## Only consider filtered synonym
import collections
value_occurrences = collections.Counter(synonym_dict.values())

filtered_synonym = {key  : value for key, value in synonym_dict.items() if value_occurrences[value] == 1}

In [ ]:
filtered_synonym

In [ ]:
mydata = mydata[['Combined Description Cleaned', 'Assignment group']] 

## Dictionary for 'Combined Description Cleaned' count
## 'Combined Description Cleaned' is 'Combined Description Cleaned' name
combined_description_count = mydata.value_counts().to_dict()
print(combined_description_count)


In [ ]:
## Get max 'Combined Description Cleaned' count to match other minority classes through data augmentation

max_combined_description_count = max(combined_description_count.items(), key=operator.itemgetter(1))[1]
print(max_combined_description_count)

In [ ]:
## Loop to interate all messages
#newdf = pd.DataFrame()
#for full_description, count in full_description_count.items() :
#    count_diff = max_full_description_count - count    ## Difference to fill
#    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
#    if (multiplication_count) :
#        old_message_df = pd.DataFrame()
#        new_message_df = pd.DataFrame()
#        for message in tqdm.tqdm(mydata[mydata["Full Description Cleaned"] == full_description]["Full Description Cleaned"]) :
#            ## Extracting existing minority class batch
#            dummy1 = pd.DataFrame([message], columns=["Full Description Cleaned"])
#            dummy1["Full Description Cleaned"] = full_description
#            old_message_df = old_message_df.append(dummy1)
#            
#            ## Creating new augmented batch from existing minority class
#            new_messages = data_augmentation_spaCy(message, multiplication_count)
#            dummy2 = pd.DataFrame(new_messages, columns=["Full Description Cleaned"])
#            dummy2["Full Description Cleaned"] = full_description
#            new_message_df = new_message_df.append(dummy2)
#        
#        ## Select random data points from augmented data
#        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
#        
#        ## Merge existing and augmented data points
#        newdf = newdf.append([old_message_df,new_message_df])
#    else :
#        newdf = newdf.append(mydata[mydata["Full Description Cleaned"] == full_description])

In [ ]:
## Loop to interate all full descriptions
newdf = pd.DataFrame()
for item, count in combined_description_count.items() :
    combined_description = item[0]
    assignment_grp = item[1]
    print(combined_description)
    print(assignment_grp)
    count_diff = max_combined_description_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(mydata[mydata["Combined Description Cleaned"] == combined_description]["Combined Description Cleaned"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame(list(zip(message, assignment_grp)), columns=['Combined Description Cleaned', 'Assignment group'])
            dummy1["Combined Description Cleaned"] = combined_description
            dummy1["Assignment group"] = assignment_grp
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation_lang_translation(message, language, multiplication_count)
            dummy2 = pd.DataFrame(list(zip(new_messages, assignment_grp)), columns=['Combined Description Cleaned', 'Assignment group'])
            dummy2["Combined Description Cleaned"] = combined_description
            dummy2["Assignment group"] = assignment_grp
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(mydata[mydata["Combined Description Cleaned"] == combined_description])

In [ ]:
newdf.describe()

In [ ]:
newdf.head(461)

In [ ]:
newdf.to_csv('datasets/spaCy_Augmented_Data.csv') 

In [ ]:
with open('datasets/spaCy_synonyms.json', 'w') as fp:
    json.dump(synonym_dict, fp, sort_keys=True, indent=4)

In [ ]:

with open('datasets/spaCy_filtered_synonyms.json', 'w') as fp:
    json.dump(filtered_synonym, fp, sort_keys=True, indent=4)